# Install packages

# General setup

In [2]:
import os # importing OS in order to make GPU visible
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# specify which device you want to work on.
# Use "-1" to work on a CPU. Default value "0" stands for the 1st GPU that will be used
os.environ["CUDA_VISIBLE_DEVICES"]='1' 

In [3]:
#code structure below courtesy https://github.com/nicknochnack/TFODCourse

In [4]:
MODEL_NAME = 'resnet50' 
MODEL_VERSION = 'v2'
PRETRAINED_MODEL_NAME = 'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz'
CLASS_LABELS_FILE = 'Class Labels.txt'
LABEL_MAP_NAME = 'label_map.pbtxt'
TFR_FILENAME = 'cleaned'
EVAL_RECORD = 'xview_eval_cleaned.record'
TRAIN_RECORD = 'xview_train_cleaned.record'
TEST_RECORD = 'xview_test_cleaned.record'
GEOJSON_FILE = 'train_data_clean.geojson'

XVIEW_UTILS_PATH = r'C:\Users\bcper\Documents\GitHub'
TF_INSTALL_PATH = r'C:\Users\bcper\Documents\GitHub\models'



In [5]:
#TODO: TFR_WORK_PATH, 
DATA_DIR_PATH = os.path.join(TF_INSTALL_PATH, 'workspace','data', '')

paths = {
    'WORK_PATH': os.path.join(TF_INSTALL_PATH, 'workspace'),
    'TFR_WORK_PATH': os.path.join(XVIEW_UTILS_PATH,'xView_data_utilities'),
    'SCRIPTS_PATH': os.path.join(TF_INSTALL_PATH,'research', 'object_detection'),
    'LABELMAP_PATH': DATA_DIR_PATH,
    'TFR_TRAIN_IMAGE': os.path.join(DATA_DIR_PATH, 'train_images',''),
    'TFR_TEST_IMAGE': os.path.join(DATA_DIR_PATH, 'test_images', ''),
    'MODEL_PATH': os.path.join(TF_INSTALL_PATH, 'workspace','models', MODEL_NAME, MODEL_VERSION, ''),
    'PRETRAINED_MODEL': os.path.join(TF_INSTALL_PATH, 'workspace','pre-trained_models'),
    'CHECKPOINT_PATH': os.path.join(TF_INSTALL_PATH, 'workspace','models', MODEL_NAME), 
    'EXPORT_PATH': os.path.join(TF_INSTALL_PATH, 'workspace','exported_models',MODEL_NAME + '_' + MODEL_VERSION),
    'PROTOC_PATH':os.path.join(TF_INSTALL_PATH,'protoc'),
    'GEOJSON_PATH': os.path.join(TF_INSTALL_PATH, 'workspace','data'),
    'CLASS_LABEL_PATH': DATA_DIR_PATH,
    'DATA_PATH': DATA_DIR_PATH
 }

In [6]:
files = {
    'PIPELINE_CONFIG_SOURCE': os.path.join(paths['PRETRAINED_MODEL'],PRETRAINED_MODEL_NAME, 'pipeline.config'), 
    'PIPELINE_CONFIG':os.path.join(TF_INSTALL_PATH, 'workspace','models', MODEL_NAME, MODEL_VERSION, 'pipeline.config'),
    'LABELMAP': os.path.join(paths['LABELMAP_PATH'], LABEL_MAP_NAME),
    'GEOJSON': os.path.join(paths['GEOJSON_PATH'], GEOJSON_FILE),
    'CLASS_LABELS': os.path.join(paths['CLASS_LABEL_PATH'], CLASS_LABELS_FILE),
    'TRAIN_RECORD': os.path.join(paths['DATA_PATH'], TRAIN_RECORD),
    'EVAL_RECORD': os.path.join(paths['DATA_PATH'], EVAL_RECORD),
    'TEST_RECORD': os.path.join(paths['DATA_PATH'], TEST_RECORD),
    'FINE_TUNE_CKPT': os.path.join(paths['PRETRAINED_MODEL'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0'),
    'TRAINING_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], 'model_main_tf2.py')
}

In [7]:
config = {
    'CLASSES': 95,
    'STEPS': 10000,
    'BATCH_SIZE': 6
}

# Generate the TF .record files

In [ ]:
os.chdir(paths['TFR_WORK_PATH'])
test_ratio = .3
TFR_TEST_FILENAME = TFR_FILENAME + 'test'


generate_train_and_eval = 'process_wv.py {} {} -s={} -t={}'.format(paths['TFR_TRAIN_IMAGE'], files['GEOJSON'], TFR_FILENAME, test_ratio)
%run {generate_train_and_eval}

generate_test = 'process_wv.py {} {} -s={} -t={}'.format(paths['TFR_TEST_IMAGE'], files['GEOJSON'], TFR_FILENAME, 0)
%run {generate_test}

# Create the label map

In [ ]:
os.chdir(paths['DATA_PATH'])
#read in the class labels
class_dict = {}
file = open(files['CLASS_LABELS'], "r")

#create a class label dictionary
for line in file:
    key, value = line.split(':')
    class_dict[int(key)] = value.strip()


#this will make the list in the correct label format for tensorflow
x = ''
for key in class_dict:
    x = x + ('item { \n\tid: ' + str(key) + '\n\tname: \"' + class_dict.get(key) + '\"\n}\n\n')
    
# Write the label map to a file
lmap = open(files['LABELMAP'],"w")#write mode
lmap.write(x)
lmap.close()

# When creating a new model using existing data created above

# Update the .config file

In [7]:
import re
import io
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

config_file = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG_SOURCE'])

In [9]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG_SOURCE'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)

    
#this is specific to the faster_rcnn model, for others we would need to change this
pipeline_config.model.faster_rcnn .num_classes = config['CLASSES']



pipeline_config.train_config.batch_size = config['BATCH_SIZE']
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(files['FINE_TUNE_CKPT'])
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_config.num_steps = config['STEPS']
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [files['TRAIN_RECORD']]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [files['EVAL_RECORD']]


config_text = text_format.MessageToString(pipeline_config)    

#create the folder if if doesnt exist
if not os.path.exists(paths['MODEL_PATH']):
    os.makedirs(paths['MODEL_PATH'])
   
    
if os.path.exists(files['PIPELINE_CONFIG']):
    with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
        f.write(config_text)
else:
    open(paths['MODEL_PATH'] + "pipeline.config", "x")
    
    with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
        f.write(config_text)



## Start the model training

In [ ]:
os.chdir(paths['SCRIPTS_PATH'])
#run the model training

train_model = '{} --pipeline_config_path={} --model_dir={}  --alsologtostderr'.format('model_main_tf2.py', files['PIPELINE_CONFIG'], paths['MODEL_PATH'])
%run {train_model}

print(train_model)

W0608 15:22:21.306470  4356 mirrored_strategy.py:379] Collective ops is not configured at program startup. Some performance features may not be enabled.


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


I0608 15:22:21.940470  4356 mirrored_strategy.py:369] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Maybe overwriting train_steps: None


I0608 15:22:21.944471  4356 config_util.py:552] Maybe overwriting train_steps: None


...

I0608 16:14:35.597526  4356 model_lib_v2.py:680] Step 8900 per-step time 0.704s loss=1.888


INFO:tensorflow:Step 9000 per-step time 0.708s loss=1.716


I0608 16:15:46.408694  4356 model_lib_v2.py:680] Step 9000 per-step time 0.708s loss=1.716


INFO:tensorflow:Step 9100 per-step time 1.350s loss=nan


I0608 16:18:01.389308  4356 model_lib_v2.py:680] Step 9100 per-step time 1.350s loss=nan


## Start the model evaluation

In [ ]:
os.chdir(paths['SCRIPTS_PATH'])
#run the model evaluation

evaluate_model = '{} --pipeline_config_path={} --model_dir={} --checkpoint_dir={}  --sample_1_of_n_eval_examples=1'.format('model_main_tf2.py', files['PIPELINE_CONFIG'], paths['MODEL_PATH'], paths['MODEL_PATH'])
%run {evaluate_model}

print(evaluate_model)

W0608 16:25:11.007423 20024 model_lib_v2.py:1063] Forced number of epochs for all eval validations to be 1.


INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1


I0608 16:25:11.009424 20024 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1


INFO:tensorflow:Maybe overwriting use_bfloat16: False


I0608 16:25:11.009424 20024 config_util.py:552] Maybe overwriting use_bfloat16: False


...

I0608 16:43:12.197861 20024 model_lib_v2.py:940] Finished eval step 8700


INFO:tensorflow:Finished eval step 8800


I0608 16:43:24.458862 20024 model_lib_v2.py:940] Finished eval step 8800


INFO:tensorflow:Finished eval step 8900


I0608 16:43:36.331861 20024 model_lib_v2.py:940] Finished eval step 8900


## Start Tensorboard

In [ ]:
os.chdir(paths['WORK_PATH'])
tensorboard_path = paths['CHECKPOINT_PATH']

!tensorboard --logdir=$tensorboard_path

print(tensorboard)

# Export Model

In [ ]:
os.chdir(paths['SCRIPTS_PATH'])

export_model = '{} --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={} --input_type=image_tensor'.format('exporter_main_v2.py', files['PIPELINE_CONFIG'], paths['MODEL_PATH'], paths['EXPORT_PATH'])
%run {export_model}

print(export_model)
